In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [ ]:
news_dataset = pd.read_csv('train.csv', encoding="unicode_escape", on_bad_lines='skip')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
from hashlib import new
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a âPoster C...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data and label
X = news_dataset.drop(columns = 'label', axis = 1)

In [ ]:
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didnât Even...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a âPoster C...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

*Stemming* :
stemming is the process of reducing a word to its root *word*

In [ ]:
port_stem = PorterStemmer()

In [26]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-z]',' ', content)
  #This replaces all characters except letters (a-z, A-Z) with spaces.
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  #Splits the string into a list of words based on whitespace.
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # Remove stopwords using for loop and Reduces words to their root form using port_stem.stem() (Porter Stemmer from nltk.stem.porter).
  stemmed_content = ' '.join(stemmed_content)
  # Joins the processed words back into a single string.
  return stemmed_content


In [27]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [28]:
news_dataset['content']

,content
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...
20798,alex ansari nato russia hold parallel exercis ...


In [29]:
#separating the data an d label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [30]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [31]:
Y.shape


(20800,)

In [32]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer() # term frequency Inverse document frequency
vectorizer.fit(X)
X = vectorizer.transform(X)

In [33]:
print(X)

  (0, 268)	0.2701012497770876
  (0, 2485)	0.36765196867972083
  (0, 2962)	0.24684501285337127
  (0, 3603)	0.3598939188262558
  (0, 3795)	0.27053324808454915
  (0, 4979)	0.23331696690935097
  (0, 7012)	0.2187416908935914
  (0, 7698)	0.24785219520671598
  (0, 8636)	0.2921251408704368
  (0, 8915)	0.36359638063260746
  (0, 13480)	0.2565896679337956
  (0, 15697)	0.2848506356272864
  (1, 1498)	0.2939891562094648
  (1, 1895)	0.15521974226349364
  (1, 2224)	0.3827320386859759
  (1, 2816)	0.19094574062359204
  (1, 3571)	0.26373768806048464
  (1, 5510)	0.7143299355715573
  (1, 6823)	0.1904660198296849
  (1, 16812)	0.30071745655510157
  (2, 2946)	0.3179886800654691
  (2, 3106)	0.46097489583229645
  (2, 5396)	0.3866530551182615
  (2, 5975)	0.3474613386728292
  (2, 9625)	0.49351492943649944
  :	:
  (20797, 3646)	0.2115550061362374
  (20797, 7049)	0.21799048897828685
  (20797, 8370)	0.22322585870464115
  (20797, 8994)	0.36160868928090795
  (20797, 9523)	0.29542040034203126
  (20797, 9593)	0.17455348

Splitting into train and test

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify = Y,random_state=2)

Training the model : Logistic Regression

In [37]:
model= LogisticRegression()

In [38]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [39]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [40]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9864783653846154


In [41]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [42]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.979326923076923


Making a predective system

In [45]:
X_new = X_test[1]
prediction =model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("The news is Real")
else:
  print("The news is Fake")

[0]
The news is Real
